In [1]:
%load_ext autoreload
%autoreload 2

from rtfs.chunk_resolution.chunk_graph import ChunkGraph
from rtfs.summarize.summarize import Summarizer
from src.config import GRAPH_ROOT
from src.chunk.chunk import chunk_repo, ChunkStrat

import json
from pathlib import Path

# AIDER
# repo_path = r"C:\Users\jpeng\Documents\projects\codesearch-backend\src\cluster\repos\aider"
# graph_path = GRAPH_ROOT / "Aider-AI_aider.git"

# graph_json = json.loads(open(graph_path, "r").read())
# cg = ChunkGraph.from_json(graph_path, graph_json)

# MOATLESS
repo_path = Path(r"C:\Users\jpeng\Documents\projects\codesearch-backend\src\cluster\repos\cowboy-server")
graph_path = GRAPH_ROOT / "aorwall_moatless-tools"

chunks = chunk_repo(repo_path, ChunkStrat.VANILLA)
# chunks_summarized = chunk_repo(repo_path, ChunkStrat.VANILLA, summarize=True)

# graph_json = json.loads(open(graph_path, "r").read())

Saving chunks to file:  C:\Users\jpeng\AppData\Local\Temp\index\cowboy-server
[Chunker]: 255 chunks used


In [2]:
# Grouped using summarized chunks

from src.chunk.lmp.summarize import summarize_lmp, CodeSummary
from rtfs.chunk_resolution.chunk_graph import ChunkGraph

cg = ChunkGraph.from_chunks(repo_path, chunks)
cg.cluster()

# for cluster in cg.get_clusters():
from rtfs.summarize.summarize import Summarizer
#     print(cluster.to_str())

Breaking cluster:  1 17
New cluster:  Exception Handling Middleware 2
New cluster:  User Authentication and Authorization 3
New cluster:  Database Utilities and Configuration 1
New cluster:  Task Queue Management 3
New cluster:  Repository Management 5
New cluster:  Target Code Management 1
New cluster:  Test Module Management 1
New cluster:  Test Generation Results 1
Colliding moves:  10
Non-colliding moves:  26


NetworkXError: The edge coverage/models.py::1-3 is not in the graph.

In [11]:
from rtfs.summarize.summarize import Summarizer

summarizer_fullcode = Summarizer(cg)
summarizer_fullcode.summarize()
graph_json = cg.to_json()

with open("full_code.json", "w") as f:
    f.write(json.dumps(graph_json, indent=2))


updating node:  10 with summary:  title='Automated Evaluation and Transition Management for Code Search and Identification' summary='This code provides a framework for evaluating different code search and identification tasks by utilizing transition rules, which dictate the progression of states during evaluation. It defines multiple evaluation functions that set up and execute these tasks, leveraging a shared Evaluation class to manage the evaluation process and generate detailed reports.' key_variables='Evaluation, evaluate_search_and_identify, search_and_code_transitions, TransitionRules'
updating node:  5 with summary:  title='Automated Evaluation and Analysis of Code Predictions' summary='This code feature manages the evaluation and analysis of code prediction experiments, generating detailed reports and managing repository contexts for different code versions. It includes functions to generate structured markdown reports, manage file contexts, handle Git repository operations, an

In [5]:
# 2024-11-09: Building the guided path
# TODO: remove overlaps in clusters

graph_json = json.loads(open("full_code.json", "r").read())
cg = ChunkGraph.from_json(graph_path, graph_json)
for cluster in cg.get_clusters():
    print(cluster.to_str())


In [27]:
chunk_paths = [
    "index/code_index.py::13",
    "index/code_index.py::7",
    "index/code_index.py::6", 
    "find/search.py::6",
    "moatless/loop.py::2",
    "moatless/loop.py::3",
    "moatless/loop.py::4",
    "moatless/transition_rules.py::1",
    "moatless/transition_rules.py::2", 
    "moatless/transition_rules.py::3",
    "moatless/transition_rules.py::4",
    "moatless/transition_rules.py::5",
    "benchmark/claude_evaluation.py::4",
    "benchmark/evaluation.py::2",
    "benchmark/evaluation.py::9",
    "benchmark/evaluation.py::5"
]

def get_cluster_from_chunk(chunks, cg):
    """Find the largest clusters containing each chunk.
    
    Args:
        chunks: List of chunk IDs to search for
        cg: ChunkGraph containing the clusters
        
    Returns:
        Set of clusters that contain the input chunks
    """
    clusters = set()
    
    for chunk in chunks:
        # Find all clusters containing this chunk
        matching_clusters = [
            cluster for cluster in cg.get_clusters(return_content=True)
            if chunk in [c.id for c in cluster.chunks]
        ]
        if matching_clusters:
            # Add the largest cluster containing this chunk
            largest_cluster = max(matching_clusters, key=lambda x: len(x.chunks))
            clusters.add(largest_cluster)
        else:
            print(f"Chunk {chunk} not found in any cluster")
            
    return clusters
        
matched_clusters = get_cluster_from_chunk(chunk_paths, cg)
for cluster in matched_clusters:
    print("Cluster: ", cluster.to_str(return_content=True))


Cluster:  Cluster 9: Automated Code Evaluation and Workflow Management
  Chunk: benchmark/evaluation.py::5
  class Evaluation:
  
      def _evaluate_instance(self, instance: dict, retry: bool = False) -> Trajectory:
          instance_id = instance["instance_id"]
          trajectory_path = os.path.join(self.trajectory_dir, f"{instance_id}.json")
          prompt_log_dir = os.path.join(self.logs_dir, f"{instance_id}")
          if not os.path.exists(prompt_log_dir):
              os.makedirs(prompt_log_dir)
  
          if os.path.exists(trajectory_path) and not retry:
              # TODO: Retry when failed or not finished?
              return Trajectory.load(trajectory_path)
  
          repo_dir = setup_swebench_repo(instance)
          persist_dir = os.path.join(self.index_store_dir, get_repo_dir_name(instance_id))
          workspace = Workspace.from_dirs(
              repo_path=repo_dir, index_dir=persist_dir, max_file_context_tokens=16000
          )
  
          problem_stat

In [20]:
for cluster in cg.get_clusters():
    print(cluster.to_str())

Cluster 10: Automated Evaluation and Transition Management for Code Search and Identification
  Chunk: benchmark/claude_evaluation.py::5
  Chunk: benchmark/claude_evaluation.py::6
  Chunk: benchmark/claude_evaluation.py::7
  Chunk: benchmark/evaluation.py::2
  Chunk: benchmark/evaluation.py::6
  Chunk: benchmark/evaluation.py::7
  Chunk: benchmark/evaluation.py::9
  Chunk: benchmark/report_v2.py::1
  Chunk: benchmark/claude_evaluation.py::1
  Chunk: benchmark/claude_evaluation.py::3
  Chunk: benchmark/claude_evaluation.py::4
  Chunk: benchmark/claude_evaluation.py::8

Cluster 5: Automated Evaluation and Analysis of Code Predictions
  Chunk: benchmark/create_dataset.py::2
  Chunk: benchmark/report_v1.py::4
  Chunk: benchmark/report_v2.py::4
  Chunk: swebench/utils.py::4
  Chunk: benchmark/utils.py::4
  Chunk: moatless/file_context.py::14
  Chunk: index/code_index.py::4
  Chunk: index/code_index.py::5
  Chunk: repository/file.py::5
  Chunk: repository/git.py::1
  Chunk: utils/repo.py::2


In [19]:
print(len([data for u, v, data in cg._graph.edges(data=True) if data["kind"] == "ClusterRef"]))
print(len([data for node, data in cg._graph.nodes(data=True) if data["kind"] == "ClusterNode"]))

274
31


In [12]:
paths = cg.get_longest_path(num_paths=None)

In [14]:
print(len(paths))
for p in paths:
    print(p.to_str())

306
Automated Codebase Initialization and Indexing Framework
-> Code Parsing and Management System for Structured Analysis
-> State-Based Code Evaluation and Transition Framework
-> Automated Code Block Processing and Transition System
-> Automated Code Evaluation and Transition Management System
-> Automated Code Change Request and Management System
Automated Codebase Initialization and Indexing Framework
-> Code Parsing and Management System for Structured Analysis
-> State-Based Code Evaluation and Transition Framework
-> Automated Code Block Processing and Transition System
-> Automated Code Change Verification and Messaging System
-> Automated Code Evaluation and Transition Management System
-> Automated Code Change Request and Management System
Automated Codebase Initialization and Indexing Framework
-> Code Parsing and Management System for Structured Analysis
-> State-Based Code Evaluation and Transition Framework
-> Automated Code Evaluation and Transition Management System
->

In [20]:
paths = loaded_cg.get_longest_path(num_paths=None)

(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 19)(19, 4)(4, 8)(8, 5)
(1, 3)(3, 19)(19, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 19)(19, 4)(4, 8)(8, 5)
(1, 3)(3, 19)(19, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 19)(19, 4)(4, 8)(8, 5)
(1, 3)(3, 19)(19, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 19)(19, 4)(4, 8)(8, 5)
(1, 3)(3, 19)(19, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 19)(19, 4)(4, 8)(8, 5)
(1, 3)(3, 19)(19, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8, 5)
(1, 3)(3, 4)(4, 8)(8

In [21]:
print(len(paths))

104


In [26]:
for p in paths:
    print(len(p))

27
26
25
25
25
24
24
24
23
23
23
23
23
22
22
22
22
22
21
21
21
21
21
20
20
20
20
19
19
19
18
17
17
17
16
15
15
15
15
15
14
14
13
13
12
12
12
11
10
10
10
10
9
9
9
8
8
8
8
8
8
8
7
7
7
7
7
6
6
6
6
6
6
6
6
6
5
5
5
5
4
4
4
4
4
4
4
4
3
3
3
3
3
3
2
2
2
2
2
2
2
2
1
1


In [7]:
for cluster in loaded_cg.get_clusters():
    print(cluster.to_str())

Cluster 1: Automated Code Transition and Evaluation Module
  Chunk: benchmark/claude_evaluation.py::1
  Chunk: benchmark/claude_evaluation.py::3
  Chunk: benchmark/claude_evaluation.py::4
  Chunk: benchmark/claude_evaluation.py::8
  Chunk: benchmark/report_v2.py::1
  Chunk: edit/clarify.py::2
  Chunk: edit/edit.py::2
  Chunk: edit/plan.py::3
  Chunk: edit/plan_lines.py::3
  Chunk: edit/review.py::4
  Chunk: find/decide.py::3
  Chunk: find/identify.py::2
  Chunk: find/search.py::6
  Chunk: moatless/loop.py::3
  Chunk: moatless/loop.py::4
  Chunk: moatless/loop.py::5
  Chunk: moatless/loop.py::6
  Chunk: moatless/loop.py::7
  Chunk: moatless/loop.py::9
  Chunk: moatless/state.py::2
  Chunk: moatless/state.py::5
  Chunk: moatless/trajectory.py::1
  Chunk: moatless/trajectory.py::6
  Chunk: moatless/transition_rules.py::1
  Chunk: moatless/transition_rules.py::2
  Chunk: moatless/transition_rules.py::4
  Chunk: moatless/transition_rules.py::5
  Chunk: moatless/transitions.py::1
  Chunk: mo

In [30]:
for c in cg_fullcode.get_clusters():
    print(len(c.chunks))

for c in cg.get_clusters():
    print(len(c.chunks))


34
8
14
5
4
2
2
10
8
15
7
29
3
2
2
13
12
6
2
2
3
7
6
4
2
2
__________________________________________
42
12
17
11
4
2
2
4
8
15
24
27
3
5
2
19
23
9
2
2
3
11
2
4
2
16


In [32]:
# 2024-11-07 TODO: compare diff between full code and summarized code
from rtfs.summarize.summarize import Summarizer

summarizer_fullcode = Summarizer(cg_fullcode)
summarizer_fullcode.summarize()


updating node:  1 with summary:  title='Agentic State Transition and Evaluation Framework' summary='This code defines a framework for handling agentic state transitions and evaluations in a software system. It includes classes and functions for defining transition rules, managing agentic states, and running evaluations based on specific parameters and conditions.' key_variables='AgenticState, TransitionRules, Evaluation, AgenticLoop'
updating node:  10 with summary:  title='Automated Evaluation and Reporting System for Code Search and Identification' summary='These code snippets define an automated evaluation system that processes and evaluates code search and identification tasks. It includes functionalities for running evaluations, processing results, generating reports, and managing transition states.' key_variables='evaluate_search_and_identify, evaluate_search_and_code, Evaluation, create_evaluation_name'
updating node:  5 with summary:  title='Automated Experiment Evaluation and 

In [77]:
for p in paths:
    print(p.to_str(verbose=False, show_refs=True))
    print("--------------------------------------------------------")

Agentic State Management and Transitions Framework
-[Content, AssistantMessage, Content, VerificationError]-> Agentic State Transition and Evaluation Framework
-[CodeBlockTypeGroup, CodeBlockType]-> Code Block Parsing and Management System
-[ReferenceScope]-> Code Relationship Management and Reference Parsing
-[ActionResponse, ActionResponse, ActionResponse, ActionResponse]-> Agentic State Management and Transitions Framework
-[Pending]-> Agentic State Transition and Evaluation Framework
-[ActionRequest]-> Agentic State Transition and Evaluation Framework
--------------------------------------------------------
Agentic State Management and Transitions Framework
-[Content, AssistantMessage, Content, VerificationError]-> Agentic State Transition and Evaluation Framework
-[CodeBlockTypeGroup, CodeBlockType]-> Code Block Parsing and Management System
-[ActionResponse, ActionResponse, ActionResponse, ActionResponse]-> Agentic State Management and Transitions Framework
-[ReferenceScope]-> Co

In [71]:
# look for vector indexing
search_paths = [p for p in paths if p.find_cluster("Code Search and Index Feature")]
print(len(search_paths))
for p in search_paths:
    print(p.to_str())
    print("___________________________________________________________________")

2
Workspace and Code Index Management for SWE-bench
[index/code_index.py::10 ] --SearchCodeResponse--> [index/types.py::2]
[index/code_index.py::6 ] --SearchCodeResponse--> [index/types.py::2]
[index/code_index.py::8 ] --SearchCodeHit--> [index/types.py::2]
[index/code_index.py::9 ] --SearchCodeResponse--> [index/types.py::2]
[index/code_index.py::7 ] --SearchCodeResponse--> [index/types.py::2]
[index/code_index.py::10 ] --SearchCodeHit--> [index/types.py::2]
[index/code_index.py::11 ] --SearchCodeResponse--> [index/types.py::2]
-> Code Search and Index Feature
[index/code_index.py::10 ] --SearchCodeResponse--> [index/types.py::2]
[index/code_index.py::6 ] --SearchCodeResponse--> [index/types.py::2]
[index/code_index.py::8 ] --SearchCodeHit--> [index/types.py::2]
[index/code_index.py::9 ] --SearchCodeResponse--> [index/types.py::2]
[index/code_index.py::7 ] --SearchCodeResponse--> [index/types.py::2]
[index/code_index.py::10 ] --SearchCodeHit--> [index/types.py::2]
[index/code_index.py